In [1]:
import numpy as np
from google.colab import drive 
drive.mount('/content/gdrive')
from sklearn.model_selection import train_test_split
import torch
import os
import torch.nn as nn

Mounted at /content/gdrive


In [55]:
%cd '/content/gdrive/MyDrive/'
print(os.getcwd())
!rm -r /content/Dataset
!mkdir /content/Dataset
!cp project_data.zip /content/Dataset/
!unzip /content/Dataset/project_data.zip -d /content/Dataset/
!rm -r /content/Dataset/project_data.zip
!rm -r /content/Dataset/__MACOSX/

/content/gdrive/MyDrive
/content/gdrive/MyDrive
rm: cannot remove '/content/Dataset': No such file or directory
Archive:  /content/Dataset/project_data.zip
   creating: /content/Dataset/project/
  inflating: /content/Dataset/project/EEG_loading.ipynb  
   creating: /content/Dataset/__MACOSX/
   creating: /content/Dataset/__MACOSX/project/
  inflating: /content/Dataset/__MACOSX/project/._EEG_loading.ipynb  
  inflating: /content/Dataset/project/X_train_valid.npy  
  inflating: /content/Dataset/__MACOSX/project/._X_train_valid.npy  
  inflating: /content/Dataset/project/.DS_Store  
  inflating: /content/Dataset/__MACOSX/project/._.DS_Store  
  inflating: /content/Dataset/project/person_train_valid.npy  
  inflating: /content/Dataset/__MACOSX/project/._person_train_valid.npy  
  inflating: /content/Dataset/project/y_train_valid.npy  
  inflating: /content/Dataset/__MACOSX/project/._y_train_valid.npy  
  inflating: /content/Dataset/project/y_test.npy  
  inflating: /content/Dataset/__MACOS

In [56]:
from scipy.ndimage import interpolation
from numpy.lib.function_base import interp
from PIL.Image import fromarray
import seaborn as sns
import matplotlib.pyplot as plt

class DatasetEEG(object):
    """
    Class for initial part of data loading , like converting labels in range 0-3
    Also has utility functions to return X,y examples to wrt to a specific person
    Splitting train data into train and validation
    """
    def __init__(self):
        pass
    
    def make_labels(self,y_train_valid):
        """
        Takes in ylabels in the range in 769 returns ylabels in range 0-3
        """
        y_train_valid=y_train_valid-769
        
        return y_train_valid
    
    def train_valid_split(self,X_train_valid,y_train_valid,person_train_valid,valsplit=0.2):
        """
        Function to split the given train_valid combination into training and validation in a 20% 80% split
        Input X_train_valid, y_train_valid, person_train_valid
        Output X_train, X_valid, y_train, y_valid, person_train, person_valid
        """
        full_size=X_train_valid.shape[0]
        val_idxs=np.random.choice(int(full_size), int(valsplit*full_size),replace=False)
        tr_idxs = np.array(list(set(range(full_size)).difference(set(val_idxs))))
        X_train=X_train_valid[tr_idxs]
        y_train=y_train_valid[tr_idxs]
        X_valid=X_train_valid[val_idxs]
        y_valid=y_train_valid[val_idxs]
        person_train=person_train_valid[tr_idxs]
        person_valid=person_train_valid[val_idxs]
        return X_train,X_valid,y_train,y_valid,person_train,person_valid
    

    def make_subject_arr(X,y,person_array,person=0):
        """
        Given a train and label arrays just return train and label arrays based on the person
        Input X,y, person_array,person
        Output X,y corresponding to that particular person
        """
        pers_idx=np.argwhere(person_array.flatten()==person)
        X_person=X[pers_idx]
        y_person=y[pers_idx]
        return X_person,y_person   
    
    def make_task_arr(self,X,y,task=0):
        """
        Given a train and label arrays just return train and label arrays based on the one of the 4 tasks
        Input X,y, task [ 0-3]
        Output X,y corresponding to that particular task
        """
        task_idx=np.argwhere(y.flatten()==task)
        X_task=X[task_idx]
        y_task=y[task_idx]
        return X_task,y_task
    
    def get_spatial_eeg_location(self):
        """
        Return the spatial layout of the sensors 
        for interpreting them as images
        
        """
        mapping =  [[ 1, 1, 0, 0, 0, 5, 5],
                    [ 1, 2, 2, 3, 4, 4, 5],
                    [ 6, 7, 8, 9,10,11,12],
                    [13,13,14,15,16,17,17],
                    [13,18,18,19,20,20,17],
                    [18,18,21,21,21,20,20]]
        return mapping
    def visualize_heatmap(self,X,y,per_arr, task=None):
        """
        
        Parameters
        ----------
        X_arr : An array of shape N, 22, 1000 (22 EEG Chhanels , 1000 time bins)
            DESCRIPTION.
        y_arr : An array of shape N,
            DESCRIPTION
        per_arr : An array os shape N,1
            DESCRIPTION.
        task : task number we want to visualize
        person : Subject number we want to visualize
            DESCRIPTION. The default is None.

        Returns
        -------
        Just visualizes the above as a heatmap
        None.
        """
        fig,ax=plt.subplots(figsize=(12,7))
     
        if not (task==None):
            task_idxs=np.argwhere(y==task).flatten()
            print(task_idxs.shape)
            idx_to_plot=np.random.choice(task_idxs)
            X_to_plt=X[idx_to_plot,:,:]
            title=" Heat map for a specific task number "+str(task)+" Sub no "+str(per_arr[idx_to_plot,0])
        else:
            idx_to_plot=np.random.choice(X.shape[0]).flatten()
            X_to_plt=X[idx_to_plot,:,:]
            title=" Heat map for a general task number  "+str(y[idx_to_plot])+ " Sub no " +str(per_arr[idx_to_plot,0])
        plt.title(title)
        #print("*"*20)
        print(X_to_plt.shape)    
        sns.heatmap(X_to_plt, cmap="rocket")
        plt.xlabel("Time bins")
        plt.ylabel("EEG channels")
        plt.show()
    def visulaize_spatial_map(self,X,y,task=1):
        """
        input: input series
        y    : output labels
        returns gif which stores all spatial maps along the time. 
        """
        import cv2
        from PIL import Image
        task_idxs=np.argwhere(y==task).flatten()
        print(task_idxs)
        idx_to_plot=np.random.choice(task_idxs)
        spatial_temporal_maps=X[idx_to_plot]
        spatial_map_resized = cv2.resize(spatial_temporal_maps,(0,0),fx = 100,fy= 100, interpolation=cv2.INTER_AREA)
        imgs = [Image.fromarray(img) for img in spatial_map_resized]
        imgs[0].save("/content/array_images.gif", save_all=True, append_images=imgs[1:], duration=500, loop=1)

    def create_images(self, X,y):
        """ Create the spatial images with 
            the help of spatial layout of sensors.
            input: X datapoints
                   y output labels
            out  : array of datapoints as images

        """
        import cv2
        from PIL import Image
        from tqdm import tqdm
        spatial_mapping =self.get_spatial_eeg_location()    
        X = np.array(X)
        time = X.shape[2]
        X_images = []
        for idx in tqdm(range(X.shape[0])):
          images_sequence =[]
          for t in range(time):
            spatial_map=np.zeros_like(spatial_mapping, dtype=np.uint8)
            x = X[idx,:,t] ## 2115, 22, 1000
            #print("x is",x.shape)
            for i in range(spatial_map.shape[0]):
              for j in range(spatial_map.shape[1]):
                if(spatial_mapping[i][j]==-1):
                  continue
                else:
                  spatial_map[i,j] = ((x[spatial_mapping[i][j]]+101.0)/201.0)*255
            images_sequence.append(spatial_map)
          X_images.append(images_sequence)
        return np.array(X_images)      
            

In [58]:
data_path='/content/Dataset/project/'

X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")
person_train_valid = np.load(data_path+"person_train_valid.npy")
X_train_valid = np.load(data_path+"X_train_valid.npy")
y_train_valid = np.load(data_path+"y_train_valid.npy")
person_test = np.load(data_path+"person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
data_loader=DatasetEEG()



y_train_valid_loaded=data_loader.make_labels(y_train_valid)
y_test_loaded = data_loader.make_labels(y_test)
X_train_valid = data_loader.create_images(X_train_valid,y_train_valid_loaded)
X_test_images = data_loader.create_images(X_test,y_test_loaded)
y_test_images = y_test_loaded

# X_train_images = np.load('/content/gdrive/MyDrive/eeg/X_train_images.npy')
# X_valid_images = np.load('/content/gdrive/MyDrive/eeg/X_valid_images.npy')
# y_train_images = np.load('/content/gdrive/MyDrive/eeg/y_train_images.npy')
# y_valid_images = np.load('/content/gdrive/MyDrive/eeg/y_valid_images.npy')
# X_test_images = np.load('/content/gdrive/MyDrive/eeg/X_test_images.npy')
# y_test_images = np.load('/content/gdrive/MyDrive/eeg/y_test_images.npy')


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


100%|██████████| 443/443 [00:42<00:00, 10.55it/s]


TypeError: ignored

In [61]:
X_train_images ,X_valid_images,  y_train_images, y_valid_images, person_train,person_valid = data_loader.train_valid_split(X_train_valid, y_train_valid_loaded, person_train_valid )


In [62]:

class Dataset(torch.utils.data.Dataset):
  def __init__(self,X,y):
    self.X=torch.from_numpy(X).float().to(device)
    self.y=torch.from_numpy(y).long().to(device)
      
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self,index):
    return self.X[index],self.y[index]


In [63]:
from torch.utils.data import DataLoader
def torch_data_loader(X_train,y_train,X_test,y_test, X_valid=[], y_valid=[],b_size=4):
  """
  torch_data loader that takes X_train,y_train .... as inputs remember X's here are of shape Numtrails*channels(22)*timebins(1000)
  we convert X to be shape of Numtrails*channels(22)*timebins(1000)*1(this dimension extra)

  This returns 3 data loaders of train,test and validation if valid is not None
  """
  # The procedure is to first call dataset and then dataloader in pytorch 
  dataset_train=Dataset(X_train_images,y_train_images)
  dataloader_train=DataLoader(dataset_train,batch_size=b_size,shuffle=True)

  dataset_test=Dataset(X_test_images,y_test_images)
  dataloader_test=DataLoader(dataset_test,batch_size=b_size,shuffle=True)

  if len(X_valid) ==0:
    dataloader_valid=[]
  else:
    dataset_valid=Dataset(X_valid,y_valid)
    dataloader_valid=DataLoader(dataset_valid,batch_size=b_size,shuffle=True)
  
  return dataloader_train,dataloader_test,dataloader_valid


In [64]:
## train epochs
import torch.nn as nn
import pandas as pd
def train_per_epoch(model,optimizer,train_loader,loss_func=nn.CrossEntropyLoss(),printevery=10,verbose=True):
  
  #put model in train mode
  model.train()
  running_loss=0
  last_loss=0
  # This will go through all the batches each entry in train_loader is a batch 
  for idx,data in enumerate(train_loader):
    inputs,labels=data

    #zero out gradients to avoid accumulation
    optimizer.zero_grad();

    # This will call .forward method of your model
    op=model(inputs)

    loss_val=loss_func(op,labels)

    # Do  backward prop
    loss_val.backward()

    #update weights
    optimizer.step()

    # loss.item gives average loss for that batch
    running_loss+=loss_val.item()

    #last_loss is the lass fof last batch
    last_loss=loss_val.item()
    # Print some info for every 10 batches
    if verbose:
      if (idx%(printevery) == 0):
        print('In training#####:batches completed={}/{}'.format(idx+1,len(train_loader)), 'The value of loss is {}'.format(running_loss/(printevery)))
        running_loss=0

  return model,last_loss

#This module would be in model architectures .py
#just putting model in test mode

def test_model(model,data_loader,loss_func=nn.CrossEntropyLoss(),print_every=10,mode="train",epoch_no=0):
  model.eval()
  # This is for each batch
  total_correct=0
  running_loss=0
  last_loss=0
  
  for idx,data in enumerate(data_loader):
    X,y=data
    output_scores=model(X)
    y_maxs,y_pred=torch.max(output_scores.data, 1)
    # Converting everything to numpy arrays 

    y_pred_np=y_pred.cpu().detach().numpy() 
    y_actual_np=y.cpu().detach().numpy()

    total_correct=total_correct+(np.where(y_pred_np==y_actual_np)[0].shape[0])

    running_loss=running_loss+loss_func(output_scores,y).item()
    last_loss=last_loss+loss_func(output_scores,y).item()
    if idx%print_every==0:
      # print('batches completed={}/{}'.format(idx+1,len(data_loader)), "The value of loss "+mode+" is {}".format(running_loss/(print_every)))
      # print('batches completed={}/{}'.format(idx+1,len(data_loader)), "The value of "+mode+" accuracy is {}".format(accuracy_score(y_actual_np,y_pred_np)))
      running_loss=0
  
  print('In testing######### epochs_completed={}'.format(epoch_no), "The value of loss "+mode+" is {}".format(last_loss/(idx+1)))
  print('In testing######### epochs completed={}'.format(epoch_no), "The value of "+mode+" accuracy is {}".format(total_correct/(len(data_loader.dataset))))
    #Returns lastloss and overall accuracy
  return  (last_loss/(idx+1)),(total_correct/(len(data_loader.dataset)))
  


# This part of the code trains the model for multiple epochs 
# For each epoch we calculate the testing accuracy and loss so that we dont over fit and identify the correct num of epochs

def train_multi_epochs(model,optimizer,all_data_loader,loss_func=nn.CrossEntropyLoss(),printevery=10,num_epochs=5):
  # We have all data loaders all we do now is train our model for all the epochs and for reach epoch calculate the validation 
  #accuracy/precision, training accuracy and precision

  dataloader_train,dataloader_test,dataloader_valid=all_data_loader
  eval_metrics={}
  eval_metrics['train_loss_hist']=[]
  eval_metrics['train_loss_accuracy']=[]
  eval_metrics['val_loss_hist']=[]
  eval_metrics['test_loss_hist']=[]
  eval_metrics['val_loss_accuracy']=[]
  eval_metrics['test_loss_accuracy']=[]
  max_test_accu=0
  best_model=None

  for i in range(num_epochs):
    model,train_loss=train_per_epoch(model,optimizer,dataloader_train,loss_func,printevery)
    
    train_loss,train_accu=test_model(model,dataloader_train,loss_func,printevery,mode="train",epoch_no=i+1)
    eval_metrics['train_loss_hist'].append(train_loss)
    eval_metrics['train_loss_accuracy'].append(train_accu)
    
    if not (len(dataloader_valid)==0):
      val_loss,val_accu=test_model(model,dataloader_valid,loss_func,printevery,mode="validation",epoch_no=i+1)
      eval_metrics['val_loss_hist'].append(val_loss)
      eval_metrics['val_loss_accuracy'].append(val_accu)
  
    test_loss,test_accu=test_model(model,dataloader_test,loss_func,printevery,mode="test",epoch_no=i+1)
    eval_metrics['test_loss_hist'].append(test_loss)
    eval_metrics['test_loss_accuracy'].append(test_accu)
    
    if test_accu > max_test_accu:
      best_model= model
      max_test_accu=test_accu
    print("Epochs done==============",i+1,"/",num_epochs)
    
  return best_model,max_test_accu,eval_metrics


def plot_train_test_curves(eval_metrics,save_path=''):
  plt.figure(1)
  plt.title("Loss vs num epochs")
  plt.ylabel("Loss")
  plt.xlabel("Epochs")
  x_range=np.arange(1,len(eval_metrics['test_loss_hist'])+1)
  plt.plot(x_range,eval_metrics['train_loss_hist'], label='Train')
  plt.plot(x_range,eval_metrics['val_loss_hist'], label='Validation')
  plt.plot(x_range,eval_metrics['test_loss_hist'], label='Test')
  plt.legend(loc='best')
  if len(save_path)>0:
    plt.savefig(save_path+'_loss.png')
  plt.show()

  plt.figure(2)
  plt.title("Accuracy vs num epochs")
  plt.ylabel("Accuracy")
  plt.xlabel("Epochs")
  x_range=np.arange(1,len(eval_metrics['test_loss_hist'])+1)
  plt.plot(x_range,eval_metrics['train_loss_accuracy'], label='Train')
  plt.plot(x_range,eval_metrics['val_loss_accuracy'], label='Validation')
  plt.plot(x_range,eval_metrics['test_loss_accuracy'], label='Test')
  plt.legend(loc='best')
  if len(save_path)>0:
    plt.savefig(save_path+'_accuracy.png')
  plt.show()

def save_eval_metrics(eval_metrics,save_path=''):
  if len(save_path)>0:
    key_list=["test_loss_hist","train_loss_hist","val_loss_hist","train_loss_accuracy","val_loss_accuracy","test_loss_accuracy"]
    op=[]
    for key in key_list:
      op.append(eval_metrics[key])
    pd.DataFrame(np.asarray(op).T,columns=key_list).to_csv(save_path+'.csv')
      
  else:
    pass


In [72]:
class CNNLayer(nn.Module):
    def __init__(self, input_channels, out_channels, kernel, stride):
        super(CNNLayer, self).__init__()
        self.convLayer = nn.Conv2d(input_channels, out_channels,kernel, stride=stride, padding=(1,1))
    def forward(self,x):
        return self.convLayer(x)

class MaxPool(nn.Module):
    def __init__(self, kernel, stride):
        super(MaxPool, self).__init__()
        self.maxpool = nn.MaxPool2d(kernel, stride=stride)
    def forward(self,x):
        return self.maxpool(x)
class CNNPipeline(nn.Module):
    def __init__(self, input_channels):
        super(CNNPipeline, self).__init__()
        self.cnn1 = CNNLayer(input_channels,16, (3,3), 1)
        self.maxPool1 = MaxPool((2,2),2)
        self.cnn2 = CNNLayer(16,32,(3,3),1)
        self.relu = nn.ReLU(inplace=True)
        #self.bn2  = nn.BatchNorm2d(32)
        #self.bn1  = nn.BatchNorm2d(16)
    def forward(self,x):
        x = self.cnn1(x)
        x = self.relu(x)
        #x  = self.bn1(x)
        x = self.maxPool1(x)
        x = self.cnn2(x)
        x = self.relu(x)
        #x  = self.bn2(x)
        # x = self.maxPool1(x)
        # x = self.cnn3(x)
        return  x
class Linear_layer(nn.Module):

    def __init__(self,hid_state=64,dropout=0.3,num_classes=4) -> None:
        super(Linear_layer,self).__init__()
        
        #Fully connected net
        self.fc_module=nn.Sequential(
            nn.Linear(hid_state,hid_state),
            nn.ReLU(inplace=True) ,
            #nn.BatchNorm1d(num_features=hid_state),
            nn.Dropout(p=dropout),
            nn.Linear(hid_state,num_classes)
        )

    def forward(self,X):
        fc_out=self.fc_module(X)
        return fc_out


class CNNLSTM(nn.Module):
    def __init__(self):
        super(CNNLSTM, self).__init__()
        self.cnnPipeline = CNNPipeline(1)
        self.lstm = torch.nn.LSTM(
            input_size= 288,
            hidden_size=64,
            num_layers=1
        )
        self.fc_module = Linear_layer(64, 0.2, 4)
        self.softmax=nn.Softmax()
    def forward(self,x):
        #print(x.size())
        batch_size, timesteps, H, W = x.size()
        cnn_in = x.view(batch_size*timesteps, 1, H, W)
        cnn_out = self.cnnPipeline(cnn_in)
        cnn_out = cnn_out.view(batch_size,timesteps,-1 )
        rnn_out,_ = self.lstm(cnn_out)
        
        rnn_out = rnn_out[:,-1,:]
        linear_out = self.fc_module(rnn_out)
        return linear_out



In [74]:
#GPU related code if GPU is available device is GPU else it is CPU
save_path = "\content"
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
b_size = 64
dataloader_train,dataloader_test,dataloader_valid=torch_data_loader(X_train_images,y_train_images,X_test_images,y_test_images,X_valid_images,y_valid_images,b_size=b_size)
# all_data_loader=(dataloader_train,dataloader_test,dataloader_valid)
all_data_loader = (dataloader_train, dataloader_test,dataloader_valid)
# all_data_loader["train"] = dataloader_train
# all_data_loader["test"] = dataloader_test
# all_data_loader["valid"] = dataloader_valid
model_cnn_lstm=CNNLSTM().to(device)
optimizer=optim.Adam(model_cnn_lstm.parameters(),lr=1e-3)
#optimizer=optim.SGD(params=model_cnn_lstm.parameters(),lr=1e-2,momentum=0.9)
best_model,max_test_accu,eval_metrics = train_multi_epochs(model_cnn_lstm,optimizer,all_data_loader,num_epochs=50,printevery=1000)
plot_train_test_curves(eval_metrics,save_path)



In training#####:batches completed=1/27 The value of loss is 0.001382137894630432
In testing######### epochs_completed=1 The value of loss train is 1.4369737527988575
In testing######### epochs completed=1 The value of train accuracy is 0.24468085106382978
In testing######### epochs_completed=1 The value of loss validation is 1.429859229496547
In testing######### epochs completed=1 The value of validation accuracy is 0.23404255319148937
In testing######### epochs_completed=1 The value of loss test is 1.4283061879021781
In testing######### epochs completed=1 The value of test accuracy is 0.2866817155756208
Epochs done============== 1 / 50
In training#####:batches completed=1/27 The value of loss is 0.0014780811071395874
In testing######### epochs_completed=2 The value of loss train is 1.3967471961621885
In testing######### epochs completed=2 The value of train accuracy is 0.2553191489361702
In testing######### epochs_completed=2 The value of loss validation is 1.3910752534866333
In test

KeyboardInterrupt: ignored

In [71]:
X_train_images[0][0]

array([[140, 140, 135, 135, 135, 134, 134],
       [140, 137, 137, 137, 132, 132, 134],
       [132, 135, 132, 132, 126, 124, 121],
       [130, 130, 128, 125, 123, 121, 121],
       [130, 124, 124, 122, 120, 120, 121],
       [124, 124, 119, 119, 119, 120, 120]], dtype=uint8)

In [76]:
print(X_train_valid)

[[[[140 140 135 ... 135 134 134]
   [140 137 137 ... 132 132 134]
   [132 135 132 ... 126 124 121]
   [130 130 128 ... 123 121 121]
   [130 124 124 ... 120 120 121]
   [124 124 119 ... 119 120 120]]

  [[134 134 130 ... 130 128 128]
   [134 133 133 ... 127 127 128]
   [130 130 128 ... 122 121 117]
   [128 128 124 ... 119 118 118]
   [128 119 119 ... 116 116 118]
   [119 119 114 ... 114 116 116]]

  [[130 130 126 ... 126 123 123]
   [130 129 129 ... 125 125 123]
   [127 127 124 ... 120 118 114]
   [124 124 119 ... 115 115 115]
   [124 114 114 ... 112 112 115]
   [114 114 109 ... 109 112 112]]

  ...

  [[118 118 115 ... 115 125 125]
   [118 116 116 ... 121 121 125]
   [125 122 125 ... 129 131 131]
   [129 129 132 ... 138 138 138]
   [129 139 139 ... 144 144 138]
   [139 139 147 ... 147 144 144]]

  [[118 118 117 ... 117 124 124]
   [118 118 118 ... 121 121 124]
   [126 121 125 ... 129 130 131]
   [128 128 133 ... 138 137 137]
   [128 140 140 ... 142 142 137]
   [140 140 143 ... 143 142 

In [77]:
X_test 

array([[[  3.27148438,  10.05859375,   1.12304688, ...,   3.75976562,
          -0.43945312,   5.6640625 ],
        [  0.73242188,   4.8828125 ,   1.12304688, ...,  -0.1953125 ,
          -3.85742188,   1.51367188],
        [  3.95507812,   9.91210938,   1.22070312, ...,   1.31835938,
          -2.78320312,   3.41796875],
        ...,
        [  9.81445312,   9.52148438,   0.04882812, ...,   4.1015625 ,
          -3.17382812,   3.22265625],
        [  8.54492188,   9.61914062,   0.87890625, ...,   6.59179688,
          -0.92773438,   4.83398438],
        [ 12.59765625,  10.88867188,  -0.63476562, ...,   5.12695312,
          -4.05273438,   2.9296875 ]],

       [[-27.34375   , -28.66210938, -24.56054688, ...,  -5.41992188,
          -4.58984375, -11.27929688],
        [-21.19140625, -21.53320312, -19.38476562, ...,  -8.83789062,
          -6.88476562, -15.8203125 ],
        [-26.22070312, -30.078125  , -26.953125  , ...,  -7.71484375,
          -5.90820312, -12.45117188],
        ...,
